In [1]:
import mdtraj as md
import numpy as np
import pandas as pd
import nglview as ng

_ColormakerRegistry()

This notebook compares the two extended active sites

In [60]:
as_d = md.load('outputs/AS_D_ext_3-Ang.pdb')
as_h = md.load('outputs/AS_H_ext_3-Ang.pdb')
               


In [61]:
as_d

<mdtraj.Trajectory with 1 frames, 603 atoms, 45 residues, without unitcells at 0x11ef51f98>

In [62]:
as_h

<mdtraj.Trajectory with 1 frames, 621 atoms, 46 residues, without unitcells at 0x11ef2efd0>

In [63]:
h_df, _ = as_h.topology.to_dataframe()
d_df, _ = as_d.topology.to_dataframe()

In [64]:
d_common_ix = d_df.merge(h_df, how='inner', on=['resSeq', 'resName', 'name']).serial_x.values
h_common_ix = d_df.merge(h_df, how='inner', on=['resSeq', 'resName', 'name']).serial_y.values

In [65]:
as_d = as_d.atom_slice(d_common_ix-1)
as_h = as_h.atom_slice(h_common_ix-1)

In [66]:
as_d = as_d.superpose(as_h)

In [68]:
as_d.save('outputs/as_d_ext-aligned.pdb')
as_h.save('outputs/as_h_ext-aligned.pdb')